In [13]:
#pip install folium

In [62]:
import pandas as pd
pd.options.display.max_rows = 15

df = pd.read_excel('Event 1-selected/2019 Event1_Analytics.xlsx')

df = df.iloc[:, [9,15]]
df = df[df['Total From Participant($)'] != 0].dropna()
df['Address  -  Participant ZIP/Postal Code'] = df['Address  -  Participant ZIP/Postal Code'].astype(str).str[:5]
df.head

<bound method NDFrame.head of      Address  -  Participant ZIP/Postal Code  Total From Participant($)
10                                     90077                       1000
12                                     89423                        460
14                                     92832                         97
15                                     91791                        600
19                                     33626                        197
...                                      ...                        ...
1900                                   90094                         50
1902                                   22205                        210
1931                                   90035                         50
1934                                   20007                         60
1936                                   92626                         60

[273 rows x 2 columns]>

In [63]:
#Generating dataset
import pgeocode

#Testing that pgeocode works

# nomi = pgeocode.Nominatim('us')
# query = nomi.query_postal_code(90077)
# print(query)

#Defining function to create associated coordinates for zipcodes

def geocode(zipcode):
    location = nomi.query_postal_code(zipcode)
    return location.latitude, location.longitude

df[['Latitude', 'Longitude']] = df['Address  -  Participant ZIP/Postal Code'].apply(lambda x: pd.Series(geocode(x)))
df

,Address - Participant ZIP/Postal Code,Total From Participant($),Latitude,Longitude
10,90077,1000,34.1112,-118.4502
12,89423,460,39.0218,-119.7314
14,92832,97,33.8680,-117.9265
15,91791,600,34.0653,-117.8978
19,33626,197,28.0509,-82.6164
...,...,...,...,...
1900,90094,50,33.9728,-118.4276
1902,22205,210,38.8836,-77.1395
1931,90035,50,34.0531,-118.3806
1934,20007,60,38.9144,-77.0740


In [75]:
import folium
from folium.plugins import MarkerCluster # for clustering the markers

import branca
import branca.colormap as cm

import math
map = folium.Map(location = [39.8283, -98.5795])

#Fitting map based on SW and NE bounds
sw = df[['Latitude', 'Longitude']].min().values.tolist()
ne = df[['Latitude', 'Longitude']].max().values.tolist()

map.fit_bounds([sw, ne]) 

#creating mapcluster object
marker_cluster = MarkerCluster().add_to(map)

#creating colormap
min_cval, max_cval = math.log(min(df['Total From Participant($)'])), math.log(max(df['Total From Participant($)']))
print(min_cval, max_cval)

#YlGnBu_09

colormap = cm.linear.PuBuGn_09.scale(min_cval, max_cval)
colormap.caption = "log(Donation Amount)"
map.add_child(colormap)

df.apply(lambda row: folium.Circle(location = [row['Latitude'], row['Longitude']],
                                  radius = 75*math.log(row['Total From Participant($)']),
                                  fill = True,
                                  opacity = 1,
                                  fill_opacity = 0.9,
                                  tooltip = folium.Tooltip('$' + str(row['Total From Participant($)']), 
                                                      permanent = True),
                                  color = colormap(math.log(row['Total From Participant($)']))
                                  ).add_to(marker_cluster),
         axis=1)

map.save('map_v2.html')
map

#tooltip='{}'.format('$' + str(row['Total From Participant($)']))

1.6094379124341003 8.950273466557375


In [40]:
print(df['Total From Participant($)'].min())
print(df['Total From Participant($)'].max())

5
7710
